In [1]:
from datasets import load_dataset

# Load the dataset 
num_examples = 20
dataset = load_dataset("librispeech_asr", "clean", split="test", streaming=True).take(num_examples)

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

import numpy as np

# process the audio data from the dataset so that it is the same length (10 seconds)
max_len = 160000
def preprocess(audio_array):
    # trim the audio to 320000 samples if it is longer than that
    if len(audio_array) > max_len:
        return audio_array[:max_len].astype(np.float32)

    else:
        # pad the shorter arrays with zeros to match the length of the longest array
        pad_len = max_len - len(audio_array)
        return np.pad(audio_array, (0, pad_len), mode='constant').astype(np.float32)


audio_data = [preprocess(example["audio"]["array"]) for example in dataset]


In [5]:
import os
from scipy.io import wavfile

def export_to_wav_folder(audio_list, output_folder):
    # Check if the output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, audio_data in enumerate(audio_list):
        # Normalize the audio data to the appropriate range for 16-bit WAV files (-32768 to 32767)
        normalized_audio = np.int16(audio_data / np.max(np.abs(audio_data)) * 32767)
        output_path = os.path.join(output_folder, f"other_{i}.wav")

        # Write the WAV file
        wavfile.write(output_path, 16000, normalized_audio) 

export_to_wav_folder(audio_data, "./MyVoice/recordings")